In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
df = pd.read_excel('Data/db_master.xlsx',sheet_name='Dist_Detail_Master')

In [3]:
colu = ['STATE NAME', 'ASM Name', 'SS Code', 'SS NAME', 'SS TOWN',
       'Dist. Code', 'Dealer Name (As Per GST)','Total Target']

In [4]:
df = df[colu]

In [5]:
df.shape

(2151, 8)

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df['STATE NAME'].unique()

array(['Gujarat', 'Haryana', 'Himachal Pradesh', 'Madhya Pradesh',
       'Maharashtra', 'Punjab', 'Rajasthan', 'Uttarakhand', nan],
      dtype=object)

In [8]:
#df = df[df['STATE NAME'].isin(['Maharashtra','Gujarat', 'Rajasthan'])]
df = df.apply(lambda col: col.str.strip().replace(r'\s+', ' ', regex=True) if col.dtype == "object" else col)

In [9]:
df['STATE NAME'] = df['STATE NAME'].str.title()
df['ASM Name'] = df['ASM Name'].str.title()
df['SS NAME'] = df['SS NAME'].str.title()
df['Dealer Name (As Per GST)'] = df['Dealer Name (As Per GST)'].str.title()

In [10]:
df['Total Target'] = df['Total Target'].astype(int,errors='ignore')
df['Total Target'] = df['Total Target'].fillna(0)
df['Total Target'] = df['Total Target'].astype(int,errors='ignore')
df['Dist. Code'] = df['Dist. Code'].fillna('NA')

In [11]:
df['SS'] = df['SS NAME']+"|"+df['SS Code']
df['DB'] = df['Dealer Name (As Per GST)']+"|"+df['Dist. Code']

In [12]:
df.rename(columns={'Total Target':'TGT','STATE NAME':'STATE','ASM Name':'ASM'},inplace=True)

In [13]:
df.columns

Index(['STATE', 'ASM', 'SS Code', 'SS NAME', 'SS TOWN', 'Dist. Code',
       'Dealer Name (As Per GST)', 'TGT', 'SS', 'DB'],
      dtype='object')

In [14]:
df.isna().sum()


STATE                       1
ASM                         1
SS Code                     1
SS NAME                     1
SS TOWN                     1
Dist. Code                  0
Dealer Name (As Per GST)    1
TGT                         0
SS                          1
DB                          1
dtype: int64

In [15]:
col2 = ['STATE','SS','ASM','DB','TGT']

In [16]:
# Google Sheet Fech data
sheet_url = "https://docs.google.com/spreadsheets/d/1Cpcj2FgA0XG8M9Rji_-DNW5LGyYmkrxEmrJ-ee58VMY/export?format=csv"
sheet = pd.read_csv(sheet_url)

In [17]:
sheet.head(5)

,Timestamp,State,SS Name,DB Name,Month,DataType,Category,Stock Id,Stock DB,Stock TGT,Week 1,Week 2,Week 3,Week 4
0,8/3/2025 11:48:00,Rajasthan,Karni Traders|ACUSDK0094,Ashish Bissa,2025-08,TGT,Bis.,Ashish Bissa,Agarwal General Store|DLR00000000174206,315,105.0,0.0,105.0,105.0
1,8/3/2025 11:48:00,Rajasthan,Karni Traders|ACUSDK0094,Ashish Bissa,2025-08,TGT,Bis.,Ashish Bissa,Agarwal Sweet|DLR00000000173474,5410,300.0,1150.0,2000.0,1960.0
2,8/3/2025 11:48:00,Rajasthan,Karni Traders|ACUSDK0094,Ashish Bissa,2025-08,TGT,Bis.,Ashish Bissa,Bajrang Agency - Napasar|DLR00000000169253,380,115.0,100.0,165.0,0.0
3,8/3/2025 11:48:00,Rajasthan,Karni Traders|ACUSDK0094,Ashish Bissa,2025-08,TGT,Bis.,Ashish Bissa,Gupta Agency - Lunkaransar|DLR00000000169270,430,170.0,120.0,0.0,140.0
4,8/3/2025 11:48:00,Rajasthan,Karni Traders|ACUSDK0094,Ashish Bissa,2025-08,TGT,Bis.,Ashish Bissa,Kripa Bherunath -|DLR00000000169281,55,25.0,0.0,30.0,0.0


In [18]:
sheet.columns

Index(['Timestamp', 'State', 'SS Name', 'DB Name', 'Month', 'DataType',
       'Category', 'Stock Id', 'Stock DB', 'Stock TGT', 'Week 1', 'Week 2',
       'Week 3', 'Week 4'],
      dtype='object')

In [19]:
sheet['dbid'] = sheet['Stock DB'].str.split('|').str[-1]


In [20]:
sheet.columns

Index(['Timestamp', 'State', 'SS Name', 'DB Name', 'Month', 'DataType',
       'Category', 'Stock Id', 'Stock DB', 'Stock TGT', 'Week 1', 'Week 2',
       'Week 3', 'Week 4', 'dbid'],
      dtype='object')

In [21]:
sheet1 = sheet[['DataType',
       'Category', 'Stock Id', 'Stock DB', 'Stock TGT', 'Week 1', 'Week 2',
       'Week 3', 'Week 4', 'dbid']]

In [22]:
df1 = df[col2]
df2 = df1
df2['dbid'] = df2['DB'].str.split('|').str[-1]


C:\Users\pc\AppData\Local\Temp\ipykernel_17292\3495708421.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['dbid'] = df2['DB'].str.split('|').str[-1]


In [23]:
merge = pd.merge(df2, sheet1,how='left',left_on='DB',right_on='Stock DB')

In [24]:
merge.columns

Index(['STATE', 'SS', 'ASM', 'DB', 'TGT', 'dbid_x', 'DataType', 'Category',
       'Stock Id', 'Stock DB', 'Stock TGT', 'Week 1', 'Week 2', 'Week 3',
       'Week 4', 'dbid_y'],
      dtype='object')

In [25]:
pvt = pd.pivot_table(data=merge,index=['STATE', 'SS', 'ASM', 'DB','dbid_x'],columns=['DataType','Category'],aggfunc={
    'Week 1':'sum',
    'Week 2':'sum',
    'Week 3':'sum',
    'Week 4':'sum',
})
pvt

Week 1  \
DataType                                                                                                               ACH   
Category                                                                                                              Bis.   
STATE       SS                             ASM           DB                                      dbid_x                      
Gujarat     Ashok Traders|ACUSDA0117       Shashi Mishra Alpha Sales|DLR00000000170375           DLR00000000170375     NaN   
                                                         Apsara Sales Agency|DLR00000000170429   DLR00000000170429     NaN   
                                                         Ashok Traders|DLR00000000170370         DLR00000000170370     NaN   
                                                         Babaramdev Sales|DLR00000000172315      DLR00000000172315     NaN   
                                                         Bhagwati Tredars|DLR00000000173281      DLR00000000173281     NaN   
...                                                                                                                    ...   
Uttarakhand Shree Ganesh Agency|ACUSDS1001 Salabh Partap Kanhaiya Kirana Store|DLR00000000160023 DLR00000000160023   110.0   
                                                         Krishna Gen Store|DLR00000000160036     DLR00000000160036  1100.0   
                                                         Shri Balaji Agencies|DLR00000000160721  DLR00000000160721   850.0   
                                                         Shri Ganesh Agencies|DLR000000174135    DLR000000174135     120.0   
                                                         Singhal Enterprises|DLR00000000160153   DLR00000000160153   120.0   

                                                                                                                          \
DataType                                                                                                                   
Category                                                                                                           Conf.   
STATE       SS                             ASM           DB                                      dbid_x                    
Gujarat     Ashok Traders|ACUSDA0117       Shashi Mishra Alpha Sales|DLR00000000170375           DLR00000000170375   NaN   
                                                         Apsara Sales Agency|DLR00000000170429   DLR00000000170429   NaN   
                                                         Ashok Traders|DLR00000000170370         DLR00000000170370   NaN   
                                                         Babaramdev Sales|DLR00000000172315      DLR00000000172315   NaN   
                                                         Bhagwati Tredars|DLR00000000173281      DLR00000000173281   NaN   
...                                                                                                                  ...   
Uttarakhand Shree Ganesh Agency|ACUSDS1001 Salabh Partap Kanhaiya Kirana Store|DLR00000000160023 DLR00000000160023   NaN   
                                                         Krishna Gen Store|DLR00000000160036     DLR00000000160036   NaN   
                                                         Shri Balaji Agencies|DLR00000000160721  DLR00000000160721   NaN   
                                                         Shri Ganesh Agencies|DLR000000174135    DLR000000174135     NaN   
                                                         Singhal Enterprises|DLR00000000160153   DLR00000000160153   NaN   

                                                                                                                            \
DataType                                                                                                                     
Category                                                                                                        

In [26]:
cols = pvt.columns

# Sort the columns as per your custom rule: TGT before ACH
sorted_cols = sorted(cols, key=lambda x: (x[0], 0 if x[1] == 'TGT' else 1, x[2]))

# Reassign sorted columns to DataFrame
pvt1 = pvt[sorted_cols]


In [27]:
pvt1.reset_index().to_excel('Data/Checklist.xlsx')

In [28]:
df1['ASM'].unique()

array(['Vacant', 'Hitesh Thakker', 'Shashi Mishra', 'Girvat Raj',
       'Hittesh Thakkar', 'Sahil Mansuri', 'Yogesh', 'Ajit Pal',
       'Rajiv Kumar', 'Mahesh', 'Jasbeer Verma', 'Badal Srivastava',
       'Rakesh Kumar', 'Pradeep Kumar', 'Parmod Kumar', 'Vinay Pandey',
       'Brajesh Sharma', 'Ankit Bhardwaj', 'Anand Gupta', '',
       'Rakesh Tripathi', 'Anil Jamadar Singh', 'Kapil Sharma',
       'Dinesh Sharma', 'Shailesh Surve', 'Neeraj Kapoor', 'Jasbir Verma',
       'Rajinder Singh', 'Shymu Verma', 'Yogesh Sharma', 'Satish Singh',
       'Puneet Kansara', 'Sarwan Sharma', 'Ajay Sachdeva / Ashu Tosh',
       'Rajendra Singh', 'Ashish', 'Subhash Yogi', 'Atul', 'Lavish Sethi',
       'Mahendra Pal Singh', 'Cp Gaur', 'Prakash Lohani', 'Avneesh',
       'Premdeep Deshwal', 'Salabh Partap', nan], dtype=object)

# Read ASM Password

In [29]:
pas = pd.read_excel('Data/asm.xlsx',usecols=['ASM','Password'])

In [30]:
df2 = df1[['STATE','SS','ASM']].drop_duplicates()
mer = pd.merge(df2,pas,on='ASM',how='left')
mer['Password'] = mer['Password'].fillna(2025)
df2 = mer

In [31]:
df3 = df1[['SS','ASM','DB','TGT']]

In [32]:
df3.sort_values(by=['DB','ASM']).to_json('Output/dbtgt.json')

In [33]:
df3.columns

Index(['SS', 'ASM', 'DB', 'TGT'], dtype='object')

# SS to ASM

In [34]:
# Create nested dictionary with password at DB level
nested = {}

for _, row in df2.iterrows():
    state = row['STATE']
    ss = row['SS']
    db = row['ASM'] # DB Name
    pwd = row['Password']

    state_dict = nested.setdefault(state, {})
    ss_dict = state_dict.setdefault(ss, {})
    db_dict = ss_dict.setdefault(db, {"password": pwd})


# Save
with open("Output/db.json", "w") as f:
    json.dump(nested, f, indent=4)

In [35]:
df2.to_excel('password.xlsx')